In [1]:
import torch
import librosa
import pandas as pd
import yaml

## Segmenting pipeline

In [2]:
with open("config/config.yml", 'r') as stream:
        config = yaml.safe_load(stream)

In [3]:
from src.Preprocessing_Pipeline import SegmentPipeline

In [4]:
seg = SegmentPipeline(config['PreProc'])

In [5]:
# seg.segment_split_pool()
# seg.build_segmented_df()

In [6]:
tp = pd.read_csv('data/data/segmented_df.csv').sample(frac=1).head(2048).reset_index().drop(columns=['index'])

In [7]:
tp

,new_uuid,old_uuid,artist
0,5e04061b-54fc-4d3b-adef-8478aa02e3f9_7,5e04061b-54fc-4d3b-adef-8478aa02e3f9,Kayne West
1,73b1d073-d067-4ad5-9de3-4bcd235c4ce7_17,73b1d073-d067-4ad5-9de3-4bcd235c4ce7,Taylor Swift
2,de8877cb-3e8e-4713-8403-e4fea7cd0a38_19,de8877cb-3e8e-4713-8403-e4fea7cd0a38,Kayne West
3,e7c5d736-fac3-4be0-805c-c74d31b5b7b5_8,e7c5d736-fac3-4be0-805c-c74d31b5b7b5,Michael jackson
4,6d9fcac7-7ca1-42ad-b795-c0882fc9e474_4,6d9fcac7-7ca1-42ad-b795-c0882fc9e474,The Beatles
...,...,...,...
2043,f0eaebce-e1ca-4547-873b-3822e2e46fe3_18,f0eaebce-e1ca-4547-873b-3822e2e46fe3,Taylor Swift
2044,df758061-7431-4f83-b93f-d1a2b17ca258_7,df758061-7431-4f83-b93f-d1a2b17ca258,David Bowie
2045,c8ed24de-c049-4dd4-9f09-e5e23c77b3de_15,c8ed24de-c049-4dd4-9f09-e5e23c77b3de,Khalid
2046,c3e20de0-3b01-4ac6-a33e-8e96b83860fc_16,c3e20de0-3b01-4ac6-a33e-8e96b83860fc,Queen


In [8]:
class Config:
    batch_size = 8
    weight_decay = 1e-8
    lr = 1e-3
    num_workers = 4
    epochs = 6
    num_classes = 24
    sr = 44100
    nmels = 256
    ROOT = "data/data/segmented_pool"
    TRAIN_AUDIO_ROOT = ROOT
    TEST_AUDIO_ROOT = ROOT
    loss_fn = torch.nn.BCEWithLogitsLoss()

In [9]:
from src.training_utils import RecoNetDataset

In [10]:
Dataset = RecoNetDataset(tp,config=Config())

In [11]:
import random as rd
Dataset.__getitem__(rd.randint(0,len(Dataset)))[0].shape

(3, 256, 866)

In [12]:
from src.training_utils import RecoNetTrainer

In [13]:
Trainer = RecoNetTrainer(4)

In [14]:
Trainer.generate_dataloaders(Dataset)

In [15]:
Trainer.train(50)

[1/50] Training loss: 2.8275	 Validation loss: 2.8079 	 train_acc: 0.2158	 val_acc: 0.1650:   2%|▏         | 1/50 [48:28<39:34:55, 2908.06s/it]

2.80789963018547


[2/50] Training loss: 2.6394	 Validation loss: 2.4766 	 train_acc: 0.2370	 val_acc: 0.2646:   4%|▍         | 2/50 [1:36:52<38:44:41, 2905.86s/it]

2.476596618740304


[3/50] Training loss: 2.3929	 Validation loss: 2.4716 	 train_acc: 0.3111	 val_acc: 0.2379:   6%|▌         | 3/50 [2:25:17<37:55:54, 2905.42s/it]

2.4716271474523452


[3/50] Training loss: 2.3929	 Validation loss: 2.4716 	 train_acc: 0.3111	 val_acc: 0.2379:   6%|▌         | 3/50 [2:47:46<43:48:27, 3355.48s/it]


KeyboardInterrupt: 